In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

### 데이터 읽기

In [2]:
# 각 기상 데이터 읽기
kr_df = pd.read_csv(r".\Git\데이터\3. 예측데이터\korea_weather(6to12).csv")
cn_df = pd.read_csv(r".\Git\데이터\3. 예측데이터\china_weather(6to12).csv")
sea_df = pd.read_csv(r".\Git\데이터\3. 예측데이터\sea_weather(6to12).csv")
mg_df = pd.read_csv(r".\Git\데이터\3. 예측데이터\mongol_weather(6to12).csv")
cn_dust = pd.read_csv(r".\Git\데이터\3. 예측데이터\중국6.6-6.11미세먼지데이터.csv", index_col = 0)

In [3]:
pm10 = pd.read_csv("./pm10_final_v1.csv", index_col = 0)

In [4]:
kr_columns = pm10.columns[5:860]
cn_columns = pm10.columns[860:1696]
sea_columns = pm10.columns[1696:2149]
mg_columns = pm10.columns[2149:2191]
cn_dust_columns = pm10.columns[2191:2205]
date_columns = pm10.columns[2205:]

### 국내 데이터 처리

In [5]:
kr_df.columns = ['지점', '일시', '기온', '강수량', '풍속', '풍향', '습도', '증기압', '이슬점온도', '현지기압', '해면기압', '지면온도']

In [6]:
kr_df

,지점,일시,기온,강수량,풍속,풍향,습도,증기압,이슬점온도,현지기압,해면기압,지면온도
0,90,2024-06-06 00:00:00,18.1,0.0,1.5,160.0,92.0,19.0,16.7,1015.2,1017.3,18.8
1,90,2024-06-06 03:00:00,18.1,0.0,2.2,310.0,91.0,18.9,16.6,1015.4,1017.5,18.2
2,90,2024-06-06 06:00:00,17.1,0.0,0.8,220.0,94.0,18.3,16.1,1015.6,1017.7,17.5
3,90,2024-06-06 09:00:00,19.7,0.0,2.1,90.0,84.0,19.1,16.8,1015.9,1017.9,23.5
4,90,2024-06-06 12:00:00,22.1,0.0,2.5,110.0,75.0,19.8,17.3,1015.5,1017.5,33.1
...,...,...,...,...,...,...,...,...,...,...,...,...
1059,279,2024-06-12 09:00:00,25.0,0.0,0.9,340.0,58.0,18.0,15.9,1001.6,1007.2,28.9
1060,279,2024-06-12 12:00:00,29.8,0.0,1.6,310.0,33.0,13.6,11.5,1001.0,1006.5,49.4
1061,279,2024-06-12 15:00:00,32.0,0.0,0.9,290.0,30.0,14.0,12.0,999.0,1004.5,52.7
1062,279,2024-06-12 18:00:00,33.0,0.0,0.6,350.0,25.0,12.5,10.3,997.7,1003.1,46.1


In [7]:
kr_df["풍향"] = kr_df["풍향"].map(lambda x: 0 if x == 360 else x)

In [8]:
kr_df["지점"] = kr_df["지점"].map(lambda x: str(x))

In [9]:
kr_pivot = kr_df.pivot(index = "일시", columns = "지점")

In [10]:
kr_pivot = kr_pivot.iloc[8:, :]

In [11]:
time_df = pd.DataFrame(kr_pivot.index)
time_df = time_df.set_index("일시")

for loc in kr_pivot["풍향"].columns:
    dummy_df = pd.get_dummies(kr_pivot["풍향"][loc])
    dummy_df.columns = dummy_df.columns.map(lambda x: ("풍향", str((int(loc), x))))
    
    time_df = pd.concat([time_df, dummy_df], axis = 1)

In [12]:
kr_pivot = kr_pivot.drop("풍향", axis = 1)

In [13]:
kr_pivot = pd.concat([kr_pivot, time_df], axis = 1)

In [14]:
kr_pivot.columns = kr_pivot.columns.map(lambda x: str(x))

In [15]:
kr_columns[~kr_columns.isin(kr_pivot.columns)]

Index(['('풍향', '(90, 20.0)')', '('풍향', '(90, 40.0)')', '('풍향', '(90, 50.0)')',
       '('풍향', '(90, 60.0)')', '('풍향', '(90, 110.0)')',
       '('풍향', '(90, 170.0)')', '('풍향', '(90, 190.0)')',
       '('풍향', '(90, 200.0)')', '('풍향', '(90, 250.0)')',
       '('풍향', '(90, 310.0)')',
       ...
       '('풍향', '(279, 100.0)')', '('풍향', '(279, 130.0)')',
       '('풍향', '(279, 140.0)')', '('풍향', '(279, 150.0)')',
       '('풍향', '(279, 160.0)')', '('풍향', '(279, 180.0)')',
       '('풍향', '(279, 190.0)')', '('풍향', '(279, 210.0)')',
       '('풍향', '(279, 230.0)')', '('풍향', '(279, 320.0)')'],
      dtype='object', length=268)

In [16]:
for column in kr_columns[~kr_columns.isin(kr_pivot.columns)]:
    kr_pivot[column] = False

C:\Users\ITSC\AppData\Local\Temp\ipykernel_15928\597790636.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  kr_pivot[column] = False
C:\Users\ITSC\AppData\Local\Temp\ipykernel_15928\597790636.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  kr_pivot[column] = False
C:\Users\ITSC\AppData\Local\Temp\ipykernel_15928\597790636.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.conc

In [17]:
kr_pivot = kr_pivot.loc[:, kr_columns]

In [18]:
sum(kr_pivot.columns != kr_columns)

0

In [19]:
kr_pivot.shape

(48, 855)

### 중국 데이터 처리

In [20]:
cn_df["지점"] = cn_df["지점"].map(lambda x: str(float(x)))

In [21]:
cn_df = cn_df.drop("지점명", axis = 1)

In [22]:
cn_df["풍향"] = cn_df["풍향"].map(lambda x: 0 if x == 360 else x)

In [23]:
cn_pivot = cn_df.pivot(index = "일시", columns = "지점")

In [26]:
cn_pivot = cn_pivot.iloc[8:, :]

In [27]:
time_df = pd.DataFrame(cn_pivot.index)
time_df = time_df.set_index("일시")

for loc in cn_pivot["풍향"].columns:
    dummy_df = pd.get_dummies(cn_pivot["풍향"][loc])
    dummy_df.columns = dummy_df.columns.map(lambda x: ("풍향", loc + "_" + str(x)))
    
    time_df = pd.concat([time_df, dummy_df], axis = 1)

In [28]:
cn_pivot = cn_pivot.drop("풍향", axis = 1)

In [29]:
cn_pivot = pd.concat([cn_pivot, time_df], axis = 1)

In [30]:
cn_pivot.columns = cn_pivot.columns.map(lambda x: str(x))

In [31]:
cn_pivot.shape

(48, 558)

In [32]:
cn_columns[~cn_columns.isin(cn_pivot.columns)]

Index(['('풍향', '50745.0_30.0')', '('풍향', '50745.0_100.0')',
       '('풍향', '50745.0_110.0')', '('풍향', '50745.0_120.0')',
       '('풍향', '50745.0_140.0')', '('풍향', '50745.0_150.0')',
       '('풍향', '50745.0_180.0')', '('풍향', '50745.0_270.0')',
       '('풍향', '50745.0_280.0')', '('풍향', '50745.0_290.0')',
       ...
       '('풍향', '59287.0_240.0')', '('풍향', '59287.0_250.0')',
       '('풍향', '59287.0_260.0')', '('풍향', '59287.0_270.0')',
       '('풍향', '59287.0_280.0')', '('풍향', '59287.0_290.0')',
       '('풍향', '59287.0_300.0')', '('풍향', '59287.0_320.0')',
       '('풍향', '59287.0_330.0')', '('풍향', '59287.0_350.0')'],
      dtype='object', length=278)

In [33]:
for column in cn_columns[~cn_columns.isin(cn_pivot.columns)]:
    cn_pivot[column] = False

C:\Users\ITSC\AppData\Local\Temp\ipykernel_15928\2132128891.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  cn_pivot[column] = False
C:\Users\ITSC\AppData\Local\Temp\ipykernel_15928\2132128891.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  cn_pivot[column] = False
C:\Users\ITSC\AppData\Local\Temp\ipykernel_15928\2132128891.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.c

In [34]:
cn_pivot = cn_pivot.loc[:, cn_columns]

In [35]:
sum(cn_pivot.columns != cn_columns)

0

In [36]:
cn_pivot.shape

(48, 836)

### 해양 데이터 처리

In [37]:
sea_columns

Index(['('풍속', '21229')', '('풍속', '22101')', '('풍속', '22102')',
       '('풍속', '22103')', '('풍속', '22104')', '('풍속', '22105')',
       '('풍속', '22106')', '('풍속', '22107')', '('풍속', '22108')',
       '('풍속', '22183')',
       ...
       '('풍향', "('22184', 280.0)")', '('풍향', "('22184', 290.0)")',
       '('풍향', "('22184', 300.0)")', '('풍향', "('22184', 310.0)")',
       '('풍향', "('22184', 320.0)")', '('풍향', "('22184', 330.0)")',
       '('풍향', "('22184', 340.0)")', '('풍향', "('22184', 350.0)")', '습도', '풍속'],
      dtype='object', length=453)

In [38]:
sea_df["지점"] = sea_df["지점"].map(lambda x: str(x))

In [39]:
sea_df["풍향"] = sea_df["풍향"].map(lambda x: 0 if x == 360 else x)

In [40]:
sea_pivot = sea_df.pivot(index = "일시", columns = "지점")

In [41]:
sea_pivot = sea_pivot.iloc[8:, :]

In [42]:
sea_pivot.head()

풍속                                                  \
지점                  21229 22101 22102 22103 22104 22105 22106 22107 22108   
일시                                                                          
2024-06-07 00:00:00   5.3   1.5   1.0   1.3   0.8   6.7   4.1   5.1   1.1   
2024-06-07 03:00:00   4.5   3.5   1.2   1.7   2.5   6.0   4.8   4.9   2.3   
2024-06-07 06:00:00   4.6   2.8   1.0   2.3   2.7   6.0   4.6   4.7   2.1   
2024-06-07 09:00:00   4.5   1.2   2.1   2.9   1.8   5.0   5.0   5.8   2.7   
2024-06-07 12:00:00   3.6   1.8   1.5   3.2   1.0   5.3   5.1   7.5   2.2   

                           ...    기온                                      \
지점                  22183  ... 22101 22102 22103 22104 22105 22106 22107   
일시                         ...                                             
2024-06-07 00:00:00   1.4  ...  16.4  18.7  19.6  19.7  20.7  20.2  19.6   
2024-06-07 03:00:00   1.6  ...  15.2  17.8  19.2  19.2  20.6  20.0  19.4   
2024-06-07 06:00:00   1.2  ...  14.6  17.1  19.2  19.1  20.4  20.1  19.7   
2024-06-07 09:00:00   1.6  ...  15.5  18.5  19.7  19.3  20.7  22.4  20.3   
2024-06-07 12:00:00   2.2  ...  15.8  19.1  20.0  20.0  21.0  22.0  20.2   

                                       
지점                  22108 22183 22184  
일시                                     
2024-06-07 00:00:00  19.5  19.2  18.2  
2024-06-07 03:00:00  18.8  18.6  17.2  
2024-06-07 06:00:00  18.4  18.9  17.0  
2024-06-07 09:00:00  18.7  19.6  18.8  
2024-06-07 12:00:00  18.8  21.4  19.1  

[5 rows x 66 columns]

In [43]:
time_df = pd.DataFrame(sea_pivot.index)
time_df = time_df.set_index("일시")

for loc in sea_pivot["풍향"].columns:
    dummy_df = pd.get_dummies(sea_pivot["풍향"][loc])
    dummy_df.columns = dummy_df.columns.map(lambda x: (("풍향", str((loc, x)))))
    
    time_df = pd.concat([time_df, dummy_df], axis = 1)

In [44]:
time_df.columns

Index([('풍향', '('21229', 140.0)'), ('풍향', '('21229', 160.0)'),
       ('풍향', '('21229', 170.0)'), ('풍향', '('21229', 180.0)'),
       ('풍향', '('21229', 190.0)'), ('풍향', '('21229', 200.0)'),
       ('풍향', '('21229', 210.0)'), ('풍향', '('21229', 220.0)'),
       ('풍향', '('21229', 230.0)'), ('풍향', '('21229', 240.0)'),
       ...
       ('풍향', '('22184', 190.0)'), ('풍향', '('22184', 200.0)'),
       ('풍향', '('22184', 210.0)'), ('풍향', '('22184', 220.0)'),
       ('풍향', '('22184', 250.0)'), ('풍향', '('22184', 280.0)'),
       ('풍향', '('22184', 290.0)'), ('풍향', '('22184', 300.0)'),
       ('풍향', '('22184', 310.0)'), ('풍향', '('22184', 340.0)')],
      dtype='object', length=194)

In [45]:
sea_pivot = sea_pivot.drop("풍향", axis = 1)

In [46]:
sea_pivot = pd.concat([sea_pivot, time_df], axis = 1)

In [47]:
sea_pivot.columns = sea_pivot.columns.map(lambda x: str(x))

In [48]:
sea_columns[~sea_columns.isin(sea_pivot.columns)]

Index(['('풍향', "('21229', 0.0)")', '('풍향', "('21229', 10.0)")',
       '('풍향', "('21229', 20.0)")', '('풍향', "('21229', 30.0)")',
       '('풍향', "('21229', 40.0)")', '('풍향', "('21229', 50.0)")',
       '('풍향', "('21229', 60.0)")', '('풍향', "('21229', 70.0)")',
       '('풍향', "('21229', 80.0)")', '('풍향', "('21229', 90.0)")',
       ...
       '('풍향', "('22184', 180.0)")', '('풍향', "('22184', 230.0)")',
       '('풍향', "('22184', 240.0)")', '('풍향', "('22184', 260.0)")',
       '('풍향', "('22184', 270.0)")', '('풍향', "('22184', 320.0)")',
       '('풍향', "('22184', 330.0)")', '('풍향', "('22184', 350.0)")', '습도', '풍속'],
      dtype='object', length=204)

In [49]:
for column in sea_columns[~sea_columns.isin(sea_pivot.columns)]:
    sea_pivot[column] = False

C:\Users\ITSC\AppData\Local\Temp\ipykernel_15928\3551806113.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  sea_pivot[column] = False
C:\Users\ITSC\AppData\Local\Temp\ipykernel_15928\3551806113.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  sea_pivot[column] = False
C:\Users\ITSC\AppData\Local\Temp\ipykernel_15928\3551806113.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd

In [50]:
sea_pivot = sea_pivot.loc[:, sea_columns]

In [51]:
sum(sea_pivot.columns != sea_columns)

0

In [52]:
sea_pivot.shape

(48, 453)

### 몽골 데이터 처리

In [53]:
mg_df = mg_df.drop(["지점", "지점명"], axis = 1)

In [54]:
mg_df = mg_df.iloc[8:, :]

In [55]:
mg_df["풍향"] = mg_df["풍향"].map(lambda x: round(x, -1))

In [56]:
mg_df = pd.concat([mg_df.drop("풍향", axis = 1), pd.get_dummies(mg_df["풍향"])], axis = 1)

In [57]:
mg_df = mg_df.set_index("일시")

In [58]:
mg_df.columns = mg_df.columns.map(lambda x: str(x))

In [59]:
mg_columns[~mg_columns.isin(mg_df.columns)]

Index(['20.0', '40.0', '90.0', '110.0', '130.0', '140.0', '150.0', '160.0',
       '170.0', '200.0', '210.0', '220.0', '240.0', '260.0', '290.0', '310.0'],
      dtype='object')

In [60]:
for column in mg_columns[~mg_columns.isin(mg_df.columns)]:
    mg_df[column] = False

In [61]:
mg_df = mg_df.loc[:, mg_columns]

In [62]:
sum(mg_df.columns != mg_columns)

0

In [63]:
mg_df.shape

(48, 42)

In [64]:
cn_dust = cn_dust.loc[:, ['베이징(PM25)', '광저우(PM25)', '난징(PM25)', '칭따오(PM25)', '상하이(PM25)',
       '우루무치(PM25)', '우한(PM25)', '베이징(PM10)', '광저우(PM10)', '난징(PM10)',
       '칭따오(PM10)', '상하이(PM10)', '우루무치(PM10)', '우한(PM10)'] ]

In [65]:
cn_dust.columns = cn_dust_columns

In [66]:
cn_dust.index = pd.date_range(start = "2024-06-06", end = "2024-06-11").date

In [67]:
final_df = pd.concat([kr_pivot, cn_pivot, sea_pivot, mg_df], axis = 1)

In [68]:
final_df.index = final_df.index.map(lambda x: datetime.strptime(x, "%Y-%m-%d %H:%M:%S"))

In [69]:
# 각 날짜별로 미세먼지 농도 값 입력
for i in range(len(final_df)):
    # 미세먼지 데이터 특정을 위한 날짜값
    date = final_df.index.date[i]
    # concat_df에 입력할 위치를 특정하기 위한 datetime값
    datetime = final_df.index[i]
    # 데이터 특정, 입력 위치 특정을 위한 컬럼명 목록
    column = cn_dust.columns
    # 데이터 입력
    final_df.loc[datetime, column] = cn_dust.loc[date, :]

KeyError: datetime.date(2024, 6, 12)

In [ ]:
final_df.shape

In [69]:
# 월 / 시간 값을 범주화한 데이터프레임 생성
datetime_df = pd.concat([pd.get_dummies(final_df.index.month, prefix = "month"), 
                         pd.get_dummies(final_df.index.hour, prefix = "hour")], axis = 1)

In [70]:
for column in date_columns[~date_columns.isin(datetime_df.columns)]:
    datetime_df[column] = False

In [71]:
datetime_df = datetime_df.loc[:, date_columns]

In [72]:
datetime_df.index = final_df.index

In [73]:
final_df = pd.concat([final_df, datetime_df], axis = 1)

In [74]:
final_df.shape

(48, 2220)

In [75]:
sum(final_df.columns == pm10.columns[5:])

2220

In [76]:
final_df.to_csv("./predict_data_v2.csv")